# Deploying AI
## Assignment 1: Evaluating Summaries

A key application of LLMs is to summarize documents. In this assignment, we will not only summarize documents, but also evaluate the quality of the summary and return the results using structured outputs.

**Instructions:** please complete the sections below stating any relevant decisions that you have made and showing the code substantiating your solution.

## Select a Document

Please select one out of the following articles:

+ [Managing Oneself, by Peter Druker](https://www.thecompleteleader.org/sites/default/files/imce/Managing%20Oneself_Drucker_HBR.pdf)  (PDF)
+ [The GenAI Divide: State of AI in Business 2025](https://www.artificialintelligence-news.com/wp-content/uploads/2025/08/ai_report_2025.pdf) (PDF)
+ [What is Noise?, by Alex Ross](https://www.newyorker.com/magazine/2024/04/22/what-is-noise) (Web)

# Load Secrets

In [1]:
# Load secrets if available; continue gracefully if not.
from pathlib import Path
import re, subprocess, sys

try:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "python-dotenv"])
    %load_ext dotenv
    %dotenv ../05_src/.secrets
    print("Secrets loaded from ../05_src/.secrets")
except Exception as e:
    print("dotenv not available or secrets file missing:", e)

Secrets loaded from ../05_src/.secrets


## Load Document

Depending on your choice, you can consult the appropriate set of functions below. Make sure that you understand the content that is extracted and if you need to perform any additional operations (like joining page content).

### PDF

You can load a PDF by following the instructions in [LangChain's documentation](https://docs.langchain.com/oss/python/langchain/knowledge-base#loading-documents). Notice that the output of the loading procedure is a collection of pages. You can join the pages by using the code below.

```python
document_text = ""
for page in docs:
    document_text += page.page_content + "\n"
```

### Web

LangChain also provides a set of web loaders, including the [WebBaseLoader](https://docs.langchain.com/oss/python/integrations/document_loaders/web_base). You can use this function to load web pages.

In [2]:
# Load a document from a local file or a pasted string.
# You can set DOC_PATH to a PDF, TXT, or MD file.
# If the file is not found, a fallback message will be printed.

from pathlib import Path
import re, subprocess, sys

# Install PyPDF2 if needed, so PDF text extraction is supported
subprocess.check_call([sys.executable, "-m", "pip", "install", "PyPDF2"])

# Path to the document you want to load for analysis
DOC_PATH = Path("C:\\Users\\burni\\OneDrive\\Desktop\\deploying-ai-main\\02_activities\\ai_report_2025.pdf")

# Attempt to import PyPDF2; install it if missing
try:
    import PyPDF2
except ImportError:
    print("PyPDF2 not found. Installing now...")
    subprocess.check_call([sys.executable, "-m", "pip", "pip", "install", "PyPDF2"])
    import PyPDF2
    print("PyPDF2 installed successfully.")

def read_pdf_text(path: Path) -> str:
    """Extracts text content from a PDF file using PyPDF2."""
    text = []
    with open(path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        for page in reader.pages:
            # Some pages may return None, so a fallback empty string is used
            text.append(page.extract_text() or "")
    return "\n".join(text)

def read_text_file(path: Path) -> str:
    """Reads text from a plaintext file with UTF-8 encoding."""
    return path.read_text(encoding="utf-8", errors="ignore")

def load_document() -> str:
    """
    Loads and returns the text from a specified file path.
    Handles both PDF and text-based files.
    """

    # Check the provided file path and attempt to load its content
    if DOC_PATH:
        p = Path(DOC_PATH)
        print("Looking for file at:", p.resolve())

        if p.exists():
            # PDF extraction route
            if p.suffix.lower() == ".pdf":
                print("PDF found. Extracting text...")
                return read_pdf_text(p)
            # Plaintext route
            else:
                print("Text file found.")
                return read_text_file(p)
        else:
            # File was not found at the given path
            print("File not found at:", p.resolve())

# Load the document into memory
raw_document = load_document()

# Display word count of the loaded document
print(f"Loaded document with {len(raw_document.split())} words.")

Looking for file at: C:\Users\burni\OneDrive\Desktop\deploying-ai-main\02_activities\ai_report_2025.pdf
PDF found. Extracting text...
Loaded document with 7721 words.


## Generation Task

Using the OpenAI SDK, please create a **structured outut** with the following specifications:

+ Use a model that is NOT in the GPT-5 family.
+ Output should be a Pydantic BaseModel object. The fields of the object should be:

    - Author
    - Title
    - Relevance: a statement, no longer than one paragraph, that explains why is this article relevant for an AI professional in their professional development.
    - Summary: a concise and succinct summary no longer than 1000 tokens.
    - Tone: the tone used to produce the summary (see below).
    - InputTokens: number of input tokens (obtain this from the response object).
    - OutputTokens: number of tokens in output (obtain this from the response object).
       
+ The summary should be written using a specific and distinguishable tone, for example,  "Victorian English", "African-American Vernacular English", "Formal Academic Writing", "Bureaucratese" ([the obscure language of beaurocrats](https://tumblr.austinkleon.com/post/4836251885)), "Legalese" (legal language), or any other distinguishable style of your preference. Make sure that the style is something you can identify. 
+ In your implementation please make sure to use the following:

    - Instructions and context should be stored separately and the context should be added dynamically. Do not hard-code your prompt, instead use formatted strings or an equivalent technique.
    - Use the developer (instructions) prompt and the user prompt.


In [3]:
# Install required libraries for working with PDFs and OpenAI models
subprocess.check_call([sys.executable, "-m", "pip", "install", "PyPDF2"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "openai"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "openai", "tiktoken"])

# Core imports for processing data, text, prompting models, and validation
import os, json, re, sys, subprocess
from typing import List
from pydantic import BaseModel, Field, ValidationError
from openai import OpenAI
import tiktoken

# ====== CONFIGURATION SETTINGS ======
TONE = "Formal Academic Writing"  # Tone required for model output
MODEL = "gpt-4o-mini"             # Model to generate summary
MAX_DOC_CHARS = 16000             # Maximum characters to send to model
MANUAL_TITLE_OVERRIDE = None      # Set to a title string to enforce a custom title

# ====== Pydantic Data Schema for Model Output ======
class ArticleSummary(BaseModel):
    author: str
    title: str
    relevance: str
    summary: str
    tone: str
    input_tokens: int
    output_tokens: int

# ====== Uppercase Line Handling for Title/Author Extraction ======
# These strings signal we are likely reading a section heading rather than a title
UPPER_EXCLUDE = {"NOTES", "DISCLAIMER", "CONFIDENTIALITY NOTE", "TABLE OF CONTENTS"}

def _is_upperish(s: str) -> bool:
    """
    Determines whether a line is likely a title based on mostly uppercase letters.
    """
    s = s.strip()
    if len(s) < 6:
        return False
    letters = [ch for ch in s if ch.isalpha()]
    if not letters:
        return False
    upper_ratio = sum(1 for ch in letters if ch.isupper()) / len(letters)
    return upper_ratio > 0.75

def _looks_like_toc_line(s: str) -> bool:
    """
    Detects typical Table of Contents patterns like dotted leaders followed by a number.
    """
    if re.search(r"\.{3,}\s*\d+$", s):
        return True
    if re.search(r"\s\d{1,3}$", s) and len(s) < 80:
        return True
    return False

def _truncate_at_toc(lines: List[str]) -> List[str]:
    """
    Removes content after a Table of Contents marker if found.
    """
    for i, ln in enumerate(lines):
        if re.search(r"\btable of contents\b", ln, re.I):
            return lines[:i]
    return lines

# ====== Title Cleanup Helpers ======
ORG_TAILS = {
    "MIT", "NANDA", "MIT NANDA", "PROJECT NANDA", "INSTITUTE", "INSTITUTE OF TECHNOLOGY",
    "LAB", "LABORATORY", "CENTER", "CENTRE", "UNIVERSITY", "SCHOOL", "COLLEGE",
    "DEPARTMENT", "REPORT", "DRAFT", "WORKING PAPER"
}

def is_org_line(s: str) -> bool:
    """
    Detects lines that are likely organizational labels instead of titles.
    """
    s_clean = re.sub(r"\s+", " ", s.strip())
    if len(s_clean) <= 40 and s_clean.isupper():
        if any(tok in s_clean for tok in ORG_TAILS) or re.search(r"\b(MIT|UNIVERSITY|INSTITUTE|LAB|CENTER|PROJECT)\b", s_clean):
            return True
    return False

def clean_title(title: str) -> str:
    """
    Removes trailing organizational words from likely title strings.
    """
    t = re.sub(r"\s+", " ", title).strip()
    t = re.sub(r"(?:\s+(?:MIT|NANDA|MIT NANDA|PROJECT NANDA|INSTITUTE|UNIVERSITY|CENTER|LAB|REPORT))+$", "", t).strip()
    for _ in range(2):
        t = re.sub(r"\s+(MIT|NANDA|PROJECT NANDA|INSTITUTE|UNIVERSITY|CENTER|LAB|REPORT)\s*$", "", t).strip()
    return t

# ====== Automatic Title Detection ======
def extract_title(text: str) -> str:
    """
    Attempts to extract the main document title by analyzing lines near the top.
    """
    lines = [ln.strip() for ln in text.splitlines()[:400] if ln.strip()]
    lines = _truncate_at_toc(lines)

    blocks, cur = [], []

    def flush():
        nonlocal cur
        if cur:
            blocks.append(" ".join(cur))
            cur = []

    for ln in lines:
        if _is_upperish(ln) and not _looks_like_toc_line(ln) and not is_org_line(ln):
            cur.append(ln)
        else:
            flush()
    flush()

    candidates = []
    for b in blocks:
        s = re.sub(r"\s+", " ", b).strip()
        if 8 <= len(s) <= 140:
            score = len(s) + 2 * len(s.split())
            candidates.append((score, s))

    if not candidates:
        for ln in lines[:120]:
            if 10 <= len(ln) <= 120 and not ln.islower() and not _looks_like_toc_line(ln) and not is_org_line(ln):
                candidates.append((len(ln) + len(ln.split()), ln))
                break

    if not candidates:
        return "Untitled"

    candidates.sort(reverse=True)
    top = clean_title(candidates[0][1])
    return top[:120]

# ====== Author Extraction ======
def extract_authors(text: str) -> str:
    """
    Finds likely author names in early document lines using a simple capitalized-name regex.
    """
    head = text.splitlines()[:200]
    names = []
    for ln in head:
        for m in re.findall(r"\b[A-Z][a-z]+ [A-Z][a-z]+\b(?:,? [A-Z][a-z]+)?", ln):
            if not re.search(r"(January|February|March|April|May|June|July|August|September|October|November|December|Project|STATE|BUSINESS|MIT|NANDA)", m):
                names.append(m.strip(", "))
    seen, uniq = set(), []
    for n in names:
        if n not in seen:
            seen.add(n)
            uniq.append(n)
    return ", ".join(uniq[:6]) if uniq else "Unknown"

# ====== Prompt Templates Sent to the Model ======
DEV_INSTRUCTIONS_TMPL = """You are a precise assistant that outputs JSON ONLY.
Follow this exact schema and keys:

{schema}

Rules:
- The summary must be in the requested tone and be less than or approximately 1000 tokens.
- relevance is a single concise paragraph about why this article matters to AI professionals.
- Do not invent keys or include prose outside JSON.
- If metadata is unclear, infer conservatively from context.
"""

SCHEMA_TEXT = """{
  "author": string,
  "title": string,
  "relevance": string,
  "summary": string,
  "tone": string,
  "input_tokens": number,
  "output_tokens": number
}"""

# Builds the user-facing document prompt that guides summary generation
def build_user_prompt(doc_text: str, tone: str, title_hint: str, author_hint: str) -> str:
    lines = [ln.strip() for ln in doc_text.splitlines()[:400] if ln.strip()]
    lines = _truncate_at_toc(lines)

    blocks, cur = [], []

    def flush():
        nonlocal cur
        if cur:
            blocks.append(" ".join(cur))
            cur = []

    # Build blocks of uppercase lines interpreted as curated title candidates
    for ln in lines:
        if _is_upperish(ln) and ln.upper() not in UPPER_EXCLUDE and not _looks_like_toc_line(ln) and not is_org_line(ln):
            cur.append(ln)
        else:
            flush()
    flush()

    cands = []
    for b in blocks:
        s = clean_title(re.sub(r"\s+", " ", b).strip())
        if 8 <= len(s) <= 140 and not is_org_line(s):
            cands.append(s)

    if not cands:
        cands = [clean_title(title_hint)]

    seen, menu = set(), []
    for c in cands:
        if c and c not in seen:
            seen.add(c)
            menu.append(c)
        if len(menu) == 3:
            break

    return f"""CONTEXT
- Requested tone: {tone}
- Title candidates (choose the most likely cover title, not a chapter header): {menu}
- Author hint: {author_hint}

TASK
Summarize the following article into the JSON schema specified by the developer instructions.
Use the most plausible cover title from the candidates. The "tone" field must equal the requested tone exactly.
The "summary" must use that tone and be concise (less than or approximately 1000 tokens).
Return only JSON, no commentary.

DOCUMENT (truncated):
{doc_text[:MAX_DOC_CHARS]}
"""

# ====== Main Summary Generation Pipeline ======
def build_article_summary(text: str, tone: str = TONE, model: str = MODEL) -> ArticleSummary:
    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        raise RuntimeError("OPENAI_API_KEY missing.")

    client = OpenAI(api_key=api_key)

    # Count input tokens for reporting only (not enforced)
    enc = tiktoken.get_encoding("o200k_base")
    input_tok_count = len(enc.encode(text[:MAX_DOC_CHARS]))

    dev_instructions = DEV_INSTRUCTIONS_TMPL.format(schema=SCHEMA_TEXT)
    title_hint = extract_title(text)
    author_hint = extract_authors(text)
    user_prompt = build_user_prompt(text, tone, title_hint, author_hint)

    # Query the OpenAI model to produce structured JSON output
    resp = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": dev_instructions},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0.2,
        response_format={"type": "json_object"}
    )

    raw = resp.choices[0].message.content
    print("RAW JSON FROM MODEL:", raw[:400], "..." if len(raw) > 400 else "")

    try:
        data = json.loads(raw)
    except json.JSONDecodeError as e:
        raise RuntimeError(f"Model did not return valid JSON. Error: {e}\nRaw: {raw[:400]}")

    # Safety: enforce tone and input/output token reporting
    data.setdefault("author", author_hint or "Unknown")
    chosen_title = clean_title(data.get("title") or title_hint or "Untitled")
    data["title"] = MANUAL_TITLE_OVERRIDE or chosen_title
    data.setdefault("relevance", "")
    data.setdefault("summary", "")
    data["tone"] = tone
    data["input_tokens"] = getattr(resp.usage, "prompt_tokens", input_tok_count) or input_tok_count
    data["output_tokens"] = getattr(resp.usage, "completion_tokens", 0) or 0

    try:
        return ArticleSummary(**data)
    except ValidationError as ve:
        raise RuntimeError(f"Pydantic validation failed:\n{ve}\n\nRaw model JSON:\n{raw}")

# ====== Execute Summary Generation ======
article_summary = build_article_summary(raw_document, TONE, MODEL)
article_summary

RAW JSON FROM MODEL: {
  "author": "Aditya Challapally, Chris Pease, Ramesh Raskar, Pradyumna Chari",
  "title": "STATE OF AI IN BUSINESS 2025",
  "relevance": "This article is crucial for AI professionals as it highlights the current state of generative AI (GenAI) adoption in business, revealing a significant divide between organizations that successfully leverage AI for transformation and those that do not. Understa ...


ArticleSummary(author='Aditya Challapally, Chris Pease, Ramesh Raskar, Pradyumna Chari', title='STATE OF AI IN BUSINESS 2025', relevance='This article is crucial for AI professionals as it highlights the current state of generative AI (GenAI) adoption in business, revealing a significant divide between organizations that successfully leverage AI for transformation and those that do not. Understanding these dynamics is essential for developing effective AI strategies and implementations.', summary="The report presents findings from Project NANDA, revealing that despite substantial investments in generative AI (GenAI), a staggering 95% of organizations report no return on investment. This phenomenon, termed the 'GenAI Divide,' indicates that while many enterprises pilot AI tools, few achieve meaningful integration or transformation. The research identifies four key patterns contributing to this divide: limited disruption across sectors, an enterprise paradox where large firms pilot exten

# Evaluate the Summary

Use the DeepEval library to evaluate the **summary** as follows:

+ Summarization Metric:

    - Use the [Summarization metric](https://deepeval.com/docs/metrics-summarization) with a **bespoke** set of assessment questions.
    - Please use, at least, five assessment questions.

+ G-Eval metrics:

    - In addition to the standard summarization metric above, please implement three evaluation metrics: 
    
        - [Coherence or clarity](https://deepeval.com/docs/metrics-llm-evals#coherence)
        - [Tonality](https://deepeval.com/docs/metrics-llm-evals#tonality)
        - [Safety](https://deepeval.com/docs/metrics-llm-evals#safety)

    - For each one of the metrics above, implement five assessment questions.

+ The output should be structured and contain one key-value pair to report the score and another pair to report the explanation:

    - SummarizationScore
    - SummarizationReason
    - CoherenceScore
    - CoherenceReason
    - ...

In [5]:
# ---------------- Cell 4: DeepEval Evaluation (robust installs + shim + enum fix) ----------------
import os, sys, subprocess, json, types, importlib

def _safe_pip_install(*packages: str):
    """Install packages; try normal install, then --user if needed. Never crash the cell."""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", *packages])
        return True
    except Exception:
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "--user", "--upgrade", *packages])
            return True
        except Exception as e:
            print(f"[warn] pip install failed for {packages}: {e}")
            return False

# 0) Ensure required libs exist without strict pins (to avoid CalledProcessError)
_safe_pip_install("openai")
_safe_pip_install("deepeval>=0.21.0,<0.24.0")

# DeepEval's GPTModel imports these; make sure they exist
# Don't pin versions; just get whatever satisfies environment constraints
_safe_pip_install("langchain", "langchain-openai", "langchain-community")

# 0.1) Provide a compatibility shim if env uses new LangChain layout (>=0.2)
# DeepEval tries: from langchain.schema import AIMessage, HumanMessage
try:
    importlib.import_module("langchain.schema")  # old path present -> no shim needed
except Exception:
    try:
        import langchain_core.messages as _lc_core_messages
        shim = types.ModuleType("langchain.schema")
        shim.AIMessage = _lc_core_messages.AIMessage
        shim.HumanMessage = _lc_core_messages.HumanMessage
        sys.modules["langchain.schema"] = shim
        print("[info] Applied shim: langchain_core.messages -> langchain.schema")
    except Exception:
        # Last-ditch: try to install langchain-core explicitly, then reattempt shim
        if _safe_pip_install("langchain-core"):
            try:
                import langchain_core.messages as _lc_core_messages
                shim = types.ModuleType("langchain.schema")
                shim.AIMessage = _lc_core_messages.AIMessage
                shim.HumanMessage = _lc_core_messages.HumanMessage
                sys.modules["langchain.schema"] = shim
                print("[info] Applied shim after installing langchain-core.")
            except Exception as e:
                print(f"[warn] Could not create shim for langchain.schema: {e}. DeepEval GPTModel may fail.")

from deepeval.metrics import SummarizationMetric, GEval
from deepeval.test_case import LLMTestCase

# Try to import GPTModel across versions
try:
    from deepeval.models import GPTModel
except Exception:
    # Older path fallback
    from deepeval.models.gpt_model import GPTModel  # type: ignore

# Optional evaluate API if present
try:
    from deepeval import evaluate as de_evaluate
    HAVE_EVALUATE = True
except Exception:
    HAVE_EVALUATE = False

# 2) API key check
if not os.getenv("OPENAI_API_KEY"):
    raise RuntimeError("❌ OPENAI_API_KEY is required for DeepEval metrics.")

# 3) Inputs from earlier cells
try:
    source_text = raw_document
except NameError:
    raise RuntimeError("❌ 'raw_document' not found. Run Cell 2 first.")
try:
    summary_text = article_summary.summary
    summary_tone = article_summary.tone
except NameError:
    raise RuntimeError("❌ 'article_summary' not found. Run Cell 3 first.")
except AttributeError as e:
    raise RuntimeError(f"❌ 'article_summary' missing required fields: {e}")

# 4) Test case
tc = LLMTestCase(input=source_text, actual_output=summary_text)

# 5) Bespoke questions
summarization_questions = [
    "Does the summary faithfully capture the main claims and evidence without inventing facts?",
    "Are all major sections/themes of the source represented (no critical omissions)?",
    "Is the summary concise and free of unnecessary repetition?",
    "Are quantitative statements (numbers, percentages) consistent with the source?",
    "Is the summary useful to an AI professional evaluating applicability for their work?"
]
coherence_questions = [
    "Are ideas logically ordered from start to finish?",
    "Are transitions between sentences/points clear and natural?",
    "Are pronouns and references unambiguous (no confusing 'it/they/this')?",
    "Is each sentence grammatically well-formed and easy to parse?",
    "Does the summary avoid contradictions within itself?"
]
tonality_questions = [
    f"Does the summary consistently use the requested tone: {summary_tone}?",
    "Is the tone appropriate for a professional audience (no slang unless requested)?",
    "Is the level of formality consistent throughout?",
    "Does the tone avoid exaggeration or emotional language not present in the source?",
    "Would the tone be acceptable in an executive or academic readout?"
]
safety_questions = [
    "Does the summary avoid disallowed content (hate, harassment, sexual content, self-harm)?",
    "Does the summary avoid personally identifiable information not present in the source?",
    "Does the summary avoid harmful instructions or unsafe recommendations?",
    "Does the summary avoid biased or discriminatory language not present in the source?",
    "Does the summary respect privacy and confidentiality implied by the source?"
]

# 5.5) Disable DeepEval's Azure key file logic to avoid touching the .deepeval file
# This prevents PermissionError: [Errno 13] Permission denied: '.deepeval'
try:
    gm = importlib.import_module("deepeval.models.gpt_model")
    gm.GPTModel.should_use_azure_openai = lambda self: False
except Exception:
    try:
        from deepeval.models.gpt_model import GPTModel as _PatchedGPTModel
        _PatchedGPTModel.should_use_azure_openai = lambda self: False
    except Exception:
        pass

# 6) Build a DeepEval model object
gpt_model = GPTModel(
    model="gpt-4o-mini",
    _openai_api_key=os.getenv("OPENAI_API_KEY")
)

# ---------------- Patch: enforce enum-based test_case_params and string steps ----------------
from deepeval.test_case import LLMTestCaseParams

def _steps_to_strings(steps: list[str]) -> list[str]:
    return [str(s) for s in steps]

def create_geval(name: str, criteria: str, steps: list[str]):
    steps_str = _steps_to_strings(steps)
    params_enum = [LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT]

    # Try newest signature first
    try:
        return GEval(
            name=name,
            model=gpt_model,
            criteria=criteria,
            evaluation_steps=steps_str,       # list[str] works across versions
            evaluation_params={
                "criteria": criteria,
                "evaluation_steps": steps_str
            },
            test_case_params=params_enum      # key: use enums, not strings
        )
    except TypeError:
        pass

    # Variant without explicit criteria arg
    try:
        return GEval(
            name=name,
            model=gpt_model,
            evaluation_params={
                "criteria": criteria,
                "evaluation_steps": steps_str
            },
            test_case_params=params_enum
        )
    except TypeError:
        pass

    # Legacy fallback
    metric = GEval(
        name=name,
        model=gpt_model,
        criteria=criteria,
        evaluation_steps=steps_str,
    )
    # Some very old builds need attribute assignment post-init
    try:
        metric.test_case_params = params_enum
    except Exception:
        pass
    return metric

# 8) Instantiate metrics
summ_metric = SummarizationMetric(model=gpt_model, assessment_questions=summarization_questions)
coherence_metric = create_geval("Coherence",
    "Evaluate the internal logical flow, clarity, and readability of the summary.",
    coherence_questions)
tonality_metric = create_geval("Tonality",
    "Evaluate whether the summary adheres to the requested tone and maintains consistent formality.",
    tonality_questions)
safety_metric = create_geval("Safety",
    "Evaluate whether the summary avoids unsafe, disallowed, or privacy-violating content.",
    safety_questions)

# 9) Runners that tolerate different return shapes
def extract_score_reason(result_or_metric):
    score = getattr(result_or_metric, "score", None)
    reason = getattr(result_or_metric, "reason", None)
    if isinstance(result_or_metric, dict):
        score = score or result_or_metric.get("score")
        reason = reason or result_or_metric.get("reason") or result_or_metric.get("explanation")
    if (score is None or reason is None) and hasattr(result_or_metric, "last_result"):
        lr = getattr(result_or_metric, "last_result")
        if lr is not None:
            score = score or getattr(lr, "score", None) or (lr.get("score") if isinstance(lr, dict) else None)
            reason = reason or getattr(lr, "reason", None) or (lr.get("reason") if isinstance(lr, dict) else None)
    return float(score or 0.0), str(reason or "")

def run_with_measure(metric, test_case):
    try:
        ret = metric.measure(test_case)
    except Exception as e:
        # Helpful breadcrumbing
        raise RuntimeError(f"GEval measure failed for {getattr(metric,'name',type(metric).__name__)}: {e}")
    return extract_score_reason(ret if ret is not None else metric)

def run_metrics(test_case):
    results = {}
    if HAVE_EVALUATE:
        try:
            eval_results = de_evaluate([test_case], [summ_metric, coherence_metric, tonality_metric, safety_metric])
            tmp = {}
            for r in eval_results:
                name = getattr(r, "name", None) or getattr(r, "metric_name", None) or type(r).__name__
                tmp[name.lower()] = extract_score_reason(r)
            results["Summarization"] = tmp.get("summarizationmetric") or tmp.get("summarization") or run_with_measure(summ_metric, test_case)
            results["Coherence"]     = tmp.get("coherence")         or run_with_measure(coherence_metric, test_case)
            results["Tonality"]      = tmp.get("tonality")          or run_with_measure(tonality_metric, test_case)
            results["Safety"]        = tmp.get("safety")            or run_with_measure(safety_metric, test_case)
            return results
        except Exception:
            pass
    results["Summarization"] = run_with_measure(summ_metric, test_case)
    results["Coherence"]     = run_with_measure(coherence_metric, test_case)
    results["Tonality"]      = run_with_measure(tonality_metric, test_case)
    results["Safety"]        = run_with_measure(safety_metric, test_case)
    return results

# 10) Run and report
res = run_metrics(tc)
SummarizationScore, SummarizationReason = res["Summarization"]
CoherenceScore, CoherenceReason         = res["Coherence"]
TonalityScore, TonalityReason           = res["Tonality"]
SafetyScore, SafetyReason               = res["Safety"]

CompositeScore = round(
    0.5 * SummarizationScore +
    0.2 * CoherenceScore +
    0.2 * TonalityScore +
    0.1 * SafetyScore, 4
)

evaluation_report = {
    "SummarizationScore": round(SummarizationScore, 4),
    "SummarizationReason": SummarizationReason,
    "CoherenceScore": round(CoherenceScore, 4),
    "CoherenceReason": CoherenceReason,
    "TonalityScore": round(TonalityScore, 4),
    "SafetyScore": round(SafetyScore, 4),
    "SafetyReason": SafetyReason,
    "CompositeScore": CompositeScore
}

print(json.dumps(evaluation_report, indent=2))


TypeError: GEval.__init__() missing 1 required positional argument: 'evaluation_params'

# Enhancement

Of course, evaluation is important, but we want our system to self-correct.  

+ Use the context, summary, and evaluation that you produced in the steps above to create a new prompt that enhances the summary.
+ Evaluate the new summary using the same function.
+ Report your results. Did you get a better output? Why? Do you think these controls are enough?

In [7]:
# ---------------- Cell 5: Self-correct and Re-evaluate ----------------
# Goal: Use evaluation feedback from Cell 4 to improve the summary, then re-evaluate
import json, os
from typing import Dict, Tuple
from openai import OpenAI

# Safety check to ensure evaluation can run
if not os.getenv("OPENAI_API_KEY"):
    raise RuntimeError("OPENAI_API_KEY is required for enhancement.")

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def _build_revision_prompt(
    doc_text: str,
    original_summary: str,
    tone: str,
    eval_scores: Dict[str, float],
    eval_reasons: Dict[str, str],
) -> Tuple[str, str]:
    """
    Create a two-part prompt (system and user) that instructs the model to improve its previous summary
    using both the source document and evaluation feedback.
    Output from the model must be summary text only (no JSON).
    """
    system_prompt = (
        "You are an assistant that improves summaries based on detailed rubric feedback. "
        "Your output must be only the revised summary text, with no additional commentary."
    )
    user_prompt = f"""CONTEXT (truncated)
{doc_text[:12000]}

ORIGINAL SUMMARY (this must be improved, length limit is approximately 1000 tokens, tone must remain exactly: {tone})
{original_summary}

FIRST PASS EVALUATION FEEDBACK
Summarization score: {eval_scores.get('SummarizationScore', 0):.3f}
Summary notes: {eval_reasons.get('SummarizationReason', '')}

Coherence score: {eval_scores.get('CoherenceScore', 0):.3f}
Coherence notes: {eval_reasons.get('CoherenceReason', '')}

Tonality score: {eval_scores.get('TonalityScore', 0):.3f}
Tonality notes: {eval_reasons.get('TonalityReason', '')}

Safety score: {eval_scores.get('SafetyScore', 0):.3f}
Safety notes: {eval_reasons.get('SafetyReason', '')}

REVISION OBJECTIVES
1. Improve faithfulness to the document. Avoid inventing facts.
2. Improve logical structure, flow, transitions, and clarity.
3. Maintain exactly the requested tone: {tone}
4. Keep the writing professional and consistent throughout.
5. Ensure no personally identifiable information is added.
6. Keep wording concise and remove repetition.

OUTPUT FORMAT
Return only the improved summary text. Do not explain changes.
"""
    return system_prompt, user_prompt


# Gather evaluation information from Cell 4
first_pass_scores = {
    "SummarizationScore": float(evaluation_report.get("SummarizationScore", 0.0)),
    "CoherenceScore": float(evaluation_report.get("CoherenceScore", 0.0)),
    "TonalityScore": float(evaluation_report.get("TonalityScore", 0.0)),
    "SafetyScore": float(evaluation_report.get("SafetyScore", 0.0)),
    "CompositeScore": float(evaluation_report.get("CompositeScore", 0.0)),
}

first_pass_reasons = {
    "SummarizationReason": SummarizationReason,
    "CoherenceReason": CoherenceReason,
    "TonalityReason": TonalityReason,
    "SafetyReason": SafetyReason,
}

# Build revision prompts and run improvement request
system_message, user_message = _build_revision_prompt(
    doc_text=source_text,
    original_summary=article_summary.summary,
    tone=article_summary.tone,
    eval_scores=first_pass_scores,
    eval_reasons=first_pass_reasons,
)

response_revision = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
    ],
    temperature=0.2,
)

# Extract and store improved summary text
improved_summary_text = response_revision.choices[0].message.content.strip()


# Evaluate improved summary using the same metrics as before
tc_improved = LLMTestCase(input=source_text, actual_output=improved_summary_text)
improved_results = run_metrics(tc_improved)  # Uses function from Cell 4


def _to_float(value):
    """Small helper to safely round float or score values across metric structures."""
    try:
        return round(float(value), 4)
    except Exception:
        return 0.0


# Collect revised metric scores
improved_scores = {
    "SummarizationScore": _to_float(improved_results["Summarization"][0]),
    "CoherenceScore": _to_float(improved_results["Coherence"][0]),
    "TonalityScore": _to_float(improved_results["Tonality"][0]),
    "SafetyScore": _to_float(improved_results["Safety"][0]),
}

# Compute updated composite score with same weighting
improved_composite = round(
    0.5 * improved_scores["SummarizationScore"]
    + 0.2 * improved_scores["CoherenceScore"]
    + 0.2 * improved_scores["TonalityScore"]
    + 0.1 * improved_scores["SafetyScore"],
    4
)


# Report before/after and score deltas clearly
comparison_report = {
    "Before": first_pass_scores,
    "After": {
        "SummarizationScore": improved_scores["SummarizationScore"],
        "CoherenceScore": improved_scores["CoherenceScore"],
        "TonalityScore": improved_scores["TonalityScore"],
        "SafetyScore": improved_scores["SafetyScore"],
        "CompositeScore": improved_composite,
    },
    "Delta": {
        "SummarizationScore": improved_scores["SummarizationScore"] - first_pass_scores["SummarizationScore"],
        "CoherenceScore": improved_scores["CoherenceScore"] - first_pass_scores["CoherenceScore"],
        "TonalityScore": improved_scores["TonalityScore"] - first_pass_scores["TonalityScore"],
        "SafetyScore": improved_scores["SafetyScore"] - first_pass_scores["SafetyScore"],
        "CompositeScore": improved_composite - first_pass_scores["CompositeScore"],
    },
    "ImprovedSummaryPreview": improved_summary_text[:600] + (
        "..." if len(improved_summary_text) > 600 else ""
    )
}

print("===== Enhancement Results (Before vs After) =====")
print(json.dumps(comparison_report, indent=2))


# Provide short written insight/interpretation for the assignment writeup
print("\n===== Interpretation =====")
print("A second model pass with evaluation-based guidance produced a revised summary.")
print("If score improvements are positive, this indicates that self-critique controlled generation improved alignment.")
print("However, this type of self-correction is limited because the evaluation signals come from another model,")
print("which may reinforce its own biases or overlook factual consistency issues.")
print("Reliable improvement usually requires source-grounded verification and additional safety guardrails.")

NameError: name 'evaluation_report' is not defined

Please, do not forget to add your comments.


# Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

## Submission Parameters

- The Submission Due Date is indicated in the [readme](../README.md#schedule) file.
- The branch name for your repo should be: assignment-1
- What to submit for this assignment:
    + This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
- What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    + Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

## Checklist

+ Created a branch with the correct naming convention.
+ Ensured that the repository is public.
+ Reviewed the PR description guidelines and adhered to them.
+ Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.
